# NBA  Project

### Import packages and begin evaluating endpoints

In [12]:
import pandas as pd

In [ ]:
from nba_api.stats.endpoints import teamyearbyyearstats

In [2]:
from nba_api.stats.static import teams
# get_teams returns a list of 30 dictionaries, each an NBA team.
nba_teams = teams.get_teams()

In [3]:
wiz = [team for team in nba_teams
         if team['full_name'] == 'Washington Wizards'][0]
wiz

{'id': 1610612764,
 'full_name': 'Washington Wizards',
 'abbreviation': 'WAS',
 'nickname': 'Wizards',
 'city': 'Washington',
 'state': 'District of Columbia',
 'year_founded': 1961}

In [4]:
wiz_id = '1610612764'
wiz_stats = teamyearbyyearstats.TeamYearByYearStats(team_id=wiz_id)
all_wiz_stats = wiz_stats.get_data_frames()[0]
all_wiz_stats

,TEAM_ID,TEAM_CITY,TEAM_NAME,YEAR,GP,WINS,LOSSES,WIN_PCT,CONF_RANK,DIV_RANK,...,OREB,DREB,REB,AST,PF,STL,TOV,BLK,PTS,PTS_RANK
0,1610612764,Chicago,Packers,1961-62,80,18,62,0.225,0,5,...,0,0,0,1802,1954,0,0,0,8874,9
1,1610612764,Chicago,Zephyrs,1962-63,80,25,55,0.313,0,5,...,0,0,0,1773,2065,0,0,0,8795,8
2,1610612764,Baltimore,Bullets,1963-64,80,31,49,0.388,0,4,...,0,0,0,1423,2073,0,0,0,8948,5
3,1610612764,Baltimore,Bullets,1964-65,80,37,43,0.463,0,3,...,0,0,0,1676,2119,0,0,0,9087,2
4,1610612764,Baltimore,Bullets,1965-66,80,38,42,0.475,0,2,...,0,0,0,1890,2199,0,0,0,9465,2
5,1610612764,Baltimore,Bullets,1966-67,81,20,61,0.247,0,5,...,0,0,0,1652,2153,0,0,0,9353,7
6,1610612764,Baltimore,Bullets,1967-68,82,36,46,0.439,0,6,...,0,0,0,1534,2127,0,0,0,9627,5
7,1610612764,Baltimore,Bullets,1968-69,82,57,25,0.695,0,1,...,0,0,0,1682,2038,0,0,0,9542,2
8,1610612764,Baltimore,Bullets,1969-70,82,50,32,0.610,0,3,...,0,0,0,1881,1896,0,0,0,9900,2
9,1610612764,Baltimore,Bullets,1970-71,82,42,40,0.512,2,1,...,0,0,0,1772,1966,0,0,0,9254,11


In [5]:
all_wiz_stats.columns

Index(['TEAM_ID', 'TEAM_CITY', 'TEAM_NAME', 'YEAR', 'GP', 'WINS', 'LOSSES',
       'WIN_PCT', 'CONF_RANK', 'DIV_RANK', 'PO_WINS', 'PO_LOSSES',
       'CONF_COUNT', 'DIV_COUNT', 'NBA_FINALS_APPEARANCE', 'FGM', 'FGA',
       'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB',
       'DREB', 'REB', 'AST', 'PF', 'STL', 'TOV', 'BLK', 'PTS', 'PTS_RANK'],
      dtype='object')

In [6]:
from nba_api.stats.endpoints import leaguedashteamstats

In [7]:
opp_scoring = leaguedashteamstats.LeagueDashTeamStats(season = '2018-19')
opp_scoring_stats = opp_scoring.get_data_frames()[0]
opp_scoring_stats

,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,...,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,CFID,CFPARAMS
0,1610612737,Atlanta Hawks,82,29,53,0.354,3971.0,3392,7524,0.451,...,30,11,14,24,29,5,12,26,10,Atlanta Hawks
1,1610612738,Boston Celtics,82,49,33,0.598,3956.0,3451,7423,0.465,...,3,4,10,2,12,26,14,6,10,Boston Celtics
2,1610612751,Brooklyn Nets,82,42,40,0.512,3996.0,3301,7358,0.449,...,26,27,28,22,23,7,15,15,10,Brooklyn Nets
3,1610612766,Charlotte Hornets,82,39,43,0.476,3966.0,3297,7362,0.448,...,2,21,18,29,3,18,19,18,10,Charlotte Hornets
4,1610612741,Chicago Bulls,82,22,60,0.268,3981.0,3266,7205,0.453,...,19,20,26,28,9,30,27,27,10,Chicago Bulls
5,1610612739,Cleveland Cavaliers,82,19,63,0.232,3951.0,3189,7184,0.444,...,9,28,30,27,6,28,29,30,10,Cleveland Cavaliers
6,1610612742,Dallas Mavericks,82,33,49,0.402,3956.0,3182,7122,0.447,...,20,29,26,6,8,2,21,20,10,Dallas Mavericks
7,1610612743,Denver Nuggets,82,54,28,0.659,3946.0,3439,7384,0.466,...,8,13,24,14,7,20,20,8,10,Denver Nuggets
8,1610612765,Detroit Pistons,82,41,41,0.500,3971.0,3185,7238,0.440,...,11,22,29,17,26,11,25,17,10,Detroit Pistons
9,1610612744,Golden State Warriors,82,57,25,0.695,3961.0,3612,7361,0.491,...,21,15,1,1,22,27,2,2,10,Golden State Warriors


In [9]:
opp_scoring_stats["SEASON"] = '2018-19'

In [10]:
opp_scoring_2 = leaguedashteamstats.LeagueDashTeamStats(season = '2019-20')
opp_scoring_stats_2 = opp_scoring_2.get_data_frames()[0]
opp_scoring_stats_2["SEASON"] = '2019-20'
opp_scoring_stats_2

,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,...,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,CFID,CFPARAMS,SEASON
0,1610612737,Atlanta Hawks,67,20,47,0.299,3256.0,2723,6067,0.449,...,20,16,30,22,21,23,29,10,Atlanta Hawks,2019-20
1,1610612738,Boston Celtics,72,48,24,0.667,3486.0,2971,6448,0.461,...,6,4,27,23,16,10,3,10,Boston Celtics,2019-20
2,1610612751,Brooklyn Nets,72,35,37,0.486,3496.0,2908,6498,0.448,...,26,20,23,18,9,17,15,10,Brooklyn Nets,2019-20
3,1610612766,Charlotte Hornets,65,23,42,0.354,3150.0,2425,5586,0.434,...,30,28,11,2,25,30,27,10,Charlotte Hornets,2019-20
4,1610612741,Chicago Bulls,65,22,43,0.338,3135.0,2573,5762,0.447,...,1,29,24,9,30,28,22,10,Chicago Bulls,2019-20
5,1610612739,Cleveland Cavaliers,65,19,46,0.292,3145.0,2619,5715,0.458,...,28,30,29,1,29,27,28,10,Cleveland Cavaliers,2019-20
6,1610612742,Dallas Mavericks,75,43,32,0.573,3635.0,3124,6772,0.461,...,27,11,7,12,4,1,6,10,Dallas Mavericks,2019-20
7,1610612743,Denver Nuggets,73,46,27,0.630,3549.0,3069,6488,0.473,...,7,18,9,16,13,12,11,10,Denver Nuggets,2019-20
8,1610612765,Detroit Pistons,66,20,46,0.303,3193.0,2595,5658,0.459,...,24,23,22,3,26,25,23,10,Detroit Pistons,2019-20
9,1610612744,Golden State Warriors,65,15,50,0.231,3145.0,2510,5730,0.438,...,18,23,8,4,26,29,30,10,Golden State Warriors,2019-20


In [13]:
team_stats = pd.DataFrame(opp_scoring_stats)


In [16]:
team_stats = team_stats.append(opp_scoring_stats_2)
team_stats

,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,...,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,CFID,CFPARAMS,SEASON
0,1610612737,Atlanta Hawks,82,29,53,0.354,3971.0,3392,7524,0.451,...,11,14,24,29,5,12,26,10,Atlanta Hawks,2018-19
1,1610612738,Boston Celtics,82,49,33,0.598,3956.0,3451,7423,0.465,...,4,10,2,12,26,14,6,10,Boston Celtics,2018-19
2,1610612751,Brooklyn Nets,82,42,40,0.512,3996.0,3301,7358,0.449,...,27,28,22,23,7,15,15,10,Brooklyn Nets,2018-19
3,1610612766,Charlotte Hornets,82,39,43,0.476,3966.0,3297,7362,0.448,...,21,18,29,3,18,19,18,10,Charlotte Hornets,2018-19
4,1610612741,Chicago Bulls,82,22,60,0.268,3981.0,3266,7205,0.453,...,20,26,28,9,30,27,27,10,Chicago Bulls,2018-19
5,1610612739,Cleveland Cavaliers,82,19,63,0.232,3951.0,3189,7184,0.444,...,28,30,27,6,28,29,30,10,Cleveland Cavaliers,2018-19
6,1610612742,Dallas Mavericks,82,33,49,0.402,3956.0,3182,7122,0.447,...,29,26,6,8,2,21,20,10,Dallas Mavericks,2018-19
7,1610612743,Denver Nuggets,82,54,28,0.659,3946.0,3439,7384,0.466,...,13,24,14,7,20,20,8,10,Denver Nuggets,2018-19
8,1610612765,Detroit Pistons,82,41,41,0.500,3971.0,3185,7238,0.440,...,22,29,17,26,11,25,17,10,Detroit Pistons,2018-19
9,1610612744,Golden State Warriors,82,57,25,0.695,3961.0,3612,7361,0.491,...,15,1,1,22,27,2,2,10,Golden State Warriors,2018-19


In [19]:
team_stats.to_csv(r'C:\Users\jakes\Documents\DAEN\Fall 2020\AIT664\team_stats.csv',index=False)

In [39]:
from nba_api.stats.endpoints import leaguedashplayerstats

In [48]:
all_stats = leaguedashplayerstats.LeagueDashPlayerStats(season='2018-19')
all_season_stats = all_stats.get_data_frames()[0]
all_season_stats["SEASON"]='2018-19'
all_stats_2 = leaguedashplayerstats.LeagueDashPlayerStats(season='2019-20')
all_season_stats_2 = all_stats_2.get_data_frames()[0]
all_season_stats_2["SEASON"]='2019-20'
all_season_stats =pd.DataFrame(all_season_stats)
all_season_stats = all_season_stats.append(all_season_stats_2)
all_season_stats

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,...,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,CFID,CFPARAMS,SEASON
0,203932,Aaron Gordon,1610612753,ORL,23.0,78,40,38,0.513,2632.533333,...,439,39,52,91,43,34,38,5,"203932,1610612753",2018-19
1,1628988,Aaron Holiday,1610612754,IND,22.0,50,31,19,0.620,645.830000,...,210,283,302,97,324,261,38,5,"1628988,1610612754",2018-19
2,1627846,Abdel Nader,1610612760,OKC,25.0,61,38,23,0.623,693.656667,...,203,351,324,464,344,261,38,5,"1627846,1610612760",2018-19
3,201143,Al Horford,1610612738,BOS,33.0,68,41,27,0.603,1972.683333,...,343,154,97,37,64,70,16,5,"201143,1610612738",2018-19
4,202329,Al-Farouq Aminu,1610612757,POR,28.0,81,52,29,0.642,2291.698333,...,382,128,133,17,95,60,38,5,"202329,1610612757",2018-19
5,1626210,Alan Williams,1610612751,BKN,26.0,5,1,4,0.200,25.533333,...,27,473,481,177,475,261,38,5,"1626210,1610612751",2018-19
6,202692,Alec Burks,1610612758,SAC,27.0,64,19,45,0.297,1374.946667,...,254,151,194,511,207,189,38,5,"202692,1610612758",2018-19
7,203518,Alex Abrines,1610612760,OKC,25.0,31,21,10,0.677,588.273333,...,174,361,367,158,385,261,38,5,"203518,1610612760",2018-19
8,1627936,Alex Caruso,1610612747,LAL,25.0,25,8,17,0.320,531.126667,...,177,283,333,136,343,131,38,5,"1627936,1610612747",2018-19
9,203458,Alex Len,1610612737,ATL,26.0,77,28,49,0.364,1544.075000,...,484,113,115,470,128,78,38,5,"203458,1610612737",2018-19


In [49]:
all_season_stats.to_csv(r'C:\Users\jakes\Documents\DAEN\Fall 2020\AIT664\all_season_stats.csv',index=False)

In [20]:
from nba_api.stats.endpoints import shotchartdetail

In [37]:
beal_id = '203078'
beal_shotchart = shotchartdetail.ShotChartDetail(player_id = beal_id, team_id = wiz_id, season_nullable='2019-20',context_measure_simple='FGA')
beal_shotchart_stats = beal_shotchart.get_data_frames()[0]

In [24]:
wiz_shotchart = shotchartdetail.ShotChartDetail(player_id = 0, team_id = wiz_id, season_nullable='2018-19',context_measure_simple='FGA')
wiz_shotchart_stats = wiz_shotchart.get_data_frames()[0]
wiz_shotchart_stats["SEASON"] = '2018-19'
wiz_shotchart_2 = shotchartdetail.ShotChartDetail(player_id = 0, team_id = wiz_id, season_nullable='2019-20',context_measure_simple='FGA')
wiz_shotchart_stats_2 = wiz_shotchart_2.get_data_frames()[0]
wiz_shotchart_stats_2["SEASON"] = '2019-20'
wiz_shot_stats = pd.DataFrame(wiz_shotchart_stats)
wiz_shot_stats = wiz_shot_stats.append(wiz_shotchart_stats_2)
wiz_shot_stats

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,SEASON
0,Shot Chart Detail,0021800015,7,202322,John Wall,1610612764,Washington Wizards,1,11,42,...,Less Than 8 ft.,0,0,8,1,1,20181018,WAS,MIA,2018-19
1,Shot Chart Detail,0021800015,10,202322,John Wall,1610612764,Washington Wizards,1,11,18,...,24+ ft.,25,150,205,1,1,20181018,WAS,MIA,2018-19
2,Shot Chart Detail,0021800015,15,202693,Markieff Morris,1610612764,Washington Wizards,1,10,48,...,8-16 ft.,10,-52,96,1,0,20181018,WAS,MIA,2018-19
3,Shot Chart Detail,0021800015,18,202322,John Wall,1610612764,Washington Wizards,1,10,12,...,8-16 ft.,15,-43,153,1,0,20181018,WAS,MIA,2018-19
4,Shot Chart Detail,0021800015,24,203078,Bradley Beal,1610612764,Washington Wizards,1,9,40,...,8-16 ft.,13,139,13,1,1,20181018,WAS,MIA,2018-19
5,Shot Chart Detail,0021800015,36,202322,John Wall,1610612764,Washington Wizards,1,8,43,...,Less Than 8 ft.,1,16,1,1,1,20181018,WAS,MIA,2018-19
6,Shot Chart Detail,0021800015,40,203078,Bradley Beal,1610612764,Washington Wizards,1,8,24,...,Less Than 8 ft.,2,26,-5,1,0,20181018,WAS,MIA,2018-19
7,Shot Chart Detail,0021800015,48,202693,Markieff Morris,1610612764,Washington Wizards,1,7,53,...,8-16 ft.,14,-76,122,1,0,20181018,WAS,MIA,2018-19
8,Shot Chart Detail,0021800015,63,203490,Otto Porter Jr.,1610612764,Washington Wizards,1,7,13,...,8-16 ft.,15,-11,152,1,0,20181018,WAS,MIA,2018-19
9,Shot Chart Detail,0021800015,69,202693,Markieff Morris,1610612764,Washington Wizards,1,6,36,...,8-16 ft.,11,95,67,1,0,20181018,WAS,MIA,2018-19


In [25]:
#Print to CSV
wiz_shot_stats.to_csv(r'C:\Users\jakes\Documents\DAEN\Fall 2020\AIT664\wiz_shot_stats.csv',index=False)

### Begin Creating Charts

In [99]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

#### Extra Possible Stats (unlikely to be needed)

In [36]:
#playerdashboardbyshootingsplits
from nba_api.stats.endpoints import playerdashboardbyshootingsplits
def_stats = playerdashboardbyshootingsplits.PlayerDashboardByShootingSplits(player_id=beal_id, season='2019-20')
all_def_stats = pd.DataFrame()
for i in range(1,7):
    all_def_stats = all_def_stats.append(def_stats.get_data_frames()[i])
    i = i+1
all_def_stats

C:\Users\jakes\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,BLKA,BLKA_RANK,CFID,CFPARAMS,EFG_PCT,EFG_PCT_RANK,FG3A,FG3A_RANK,FG3M,FG3M_RANK,...,PCT_AST_3PM,PCT_AST_3PM_RANK,PCT_AST_FGM,PCT_AST_FGM_RANK,PCT_UAST_2PM,PCT_UAST_2PM_RANK,PCT_UAST_3PM,PCT_UAST_3PM_RANK,PCT_UAST_FGM,PCT_UAST_FGM_RANK
0,32,9.0,47,Less Than 5 ft.,0.639,1.0,0,6.0,0,4.0,...,0.000,3.0,0.418,3.0,0.582,5.0,0.000,4.0,0.582,5.0
1,12,8.0,47,5-9 ft.,0.356,6.0,0,6.0,0,4.0,...,0.000,3.0,0.216,6.0,0.784,1.0,0.000,4.0,0.784,2.0
2,6,7.0,47,10-14 ft.,0.453,4.0,0,6.0,0,4.0,...,0.000,3.0,0.306,5.0,0.694,3.0,0.000,4.0,0.694,3.0
3,1,5.0,47,15-19 ft.,0.400,5.0,0,6.0,0,4.0,...,0.000,3.0,0.387,4.0,0.613,4.0,0.000,4.0,0.613,4.0
4,0,1.0,47,20-24 ft.,0.490,3.0,115,2.0,41,2.0,...,0.707,1.0,0.615,2.0,0.727,2.0,0.293,3.0,0.385,6.0
5,1,5.0,47,25-29 ft.,0.536,2.0,358,1.0,128,1.0,...,0.617,2.0,0.617,1.0,0.000,6.0,0.383,2.0,0.383,7.0
6,0,1.0,47,30-34 ft.,0.250,7.0,6,3.0,1,3.0,...,0.000,3.0,0.000,7.0,0.000,6.0,1.000,1.0,1.000,1.0
7,0,1.0,47,35-39 ft.,0.000,8.0,1,4.0,0,4.0,...,0.000,3.0,0.000,7.0,0.000,6.0,0.000,4.0,0.000,8.0
8,0,1.0,47,40+ ft.,0.000,8.0,1,4.0,0,4.0,...,0.000,3.0,0.000,7.0,0.000,6.0,0.000,4.0,0.000,8.0
0,43,5.0,48,Less Than 8 ft.,0.592,1.0,0,3.0,0,2.0,...,0.000,2.0,0.399,2.0,0.601,3.0,0.000,2.0,0.601,3.0
